# 1.
# Aries Basic Controller - Alice

This notebook walks through a MVP for PryVote assuming the Openminded community will be issued an Openmined commuity credential that will enable community members to vote on basic things. The credential will be issued across a previously established connection.

It is best run in parallel with the [openmind-pryvote-voter notebook](http://127.0.0.1:8889/notebooks/openmind-pryvote-voter.ipynb) controlling the agent receiving the credential to enable voting.

If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)


In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
alice_agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)


# 2
## Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(alice_agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
alice_agent_controller.register_listeners([cred_listener], defaults=True)


# 3
## Check the agent has an active connection

**Note: An active connection is required, this should have been established on start up through the python script create_connection.py in the setup folder. If not it is possible to run through the did-exchange tutorial to create one between Alice and Bob**

* [Alice](http://localhost:8888/notebooks/did-exchange-inviter.ipynb)
* [Bob](http://localhost:8889/notebooks/did-exchange-invitee.ipynb)

In [18]:
try:
    response = await alice_agent_controller.connections.get_connections()
    results = response['results']
    print("Results : ", results)
    print('\n')
    if len(results) > 0:
        connection = response['results'][0]
        print("Connection :", connection)
        if connection['state'] == 'active':
            connection_id = connection["connection_id"]
            alice_connection_id = connection_id
            print("\nActive Connection ID : ", connection_id)
        else:
            print("\nNo active connection found - wait a bit and execute again")
    else:
        print("You must create a connection")
except ConnectionRefusedError as e:
    print(repr(e))

Results :  [{'my_did': '5GaQxuSKGe2iSSv3cMHNRr', 'invitation_key': '8rqmwfahoEDDYWvvQDfW5FwP8jB65xZYCnVtbdKc5BtB', 'request_id': '019fe144-3daf-435d-8774-5ada7c538ce3', 'accept': 'manual', 'routing_state': 'none', 'updated_at': '2020-09-01 20:41:58.473911Z', 'initiator': 'external', 'state': 'active', 'connection_id': 'ac91d05a-f26c-487f-8cbc-95c6da689089', 'their_did': 'FJ9sRat4y2CVaL6CA22Uqf', 'invitation_mode': 'once', 'their_label': 'Bob', 'created_at': '2020-09-01 20:41:46.258467Z'}]


Connection : {'my_did': '5GaQxuSKGe2iSSv3cMHNRr', 'invitation_key': '8rqmwfahoEDDYWvvQDfW5FwP8jB65xZYCnVtbdKc5BtB', 'request_id': '019fe144-3daf-435d-8774-5ada7c538ce3', 'accept': 'manual', 'routing_state': 'none', 'updated_at': '2020-09-01 20:41:58.473911Z', 'initiator': 'external', 'state': 'active', 'connection_id': 'ac91d05a-f26c-487f-8cbc-95c6da689089', 'their_did': 'FJ9sRat4y2CVaL6CA22Uqf', 'invitation_mode': 'once', 'their_label': 'Bob', 'created_at': '2020-09-01 20:41:46.258467Z'}

Active Co

# 6
## Write a Schema to the Ledger

For more details see the [schema-api notebook](http://localhost:8888/notebooks/schema_api.ipynb)

In [19]:
# Define you schema name - must be unique on the ledger
schema_name = "generic_person"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["fullname", "surname", "email", "age"]

response = await alice_agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)


PQRXDxdGqQGSZ8z69p4xZP:2:generic_person:0.0.1


# 7
## Write a Credential Definition to the Ledger

More details in the [definitions notebook](http://localhost:8888/notebooks/definitions_api.ipynb)

In [20]:
response = await alice_agent_controller.definitions.write_cred_def(schema_id)
print(response)
cred_def_id = response["credential_definition_id"]
print(cred_def_id)

{'credential_definition_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default'}
PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default


**Note: You should be able to see both schema and definition transactions on the local network [here](http://localhost:9000)**

# 8
## Populate the Attribues to Issue to Bob

The schema defines two attributes: name and community_member. Both must be populated with attribute values that Openmined Community (Alice) wishes to issue in a credential to Openmind-pryvote-voter (Bob). To do this a list of objects must be created, each object containing the name of the attribute and it's value at a minimum. You can set the values to anything you wish.

TODO: Some additional fields such as mime-type can be defined.

In [21]:
credential_attributes = [
    {"name": "fullname", "value": "Bob"},
    {"name": "surname", "value": "Bobber"},
    {"name": "email", "value": "bob@bobber.org"},
    {"name": "age", "value": "21"},
]

print(credential_attributes)

[{'name': 'fullname', 'value': 'Bob'}, {'name': 'surname', 'value': 'Bobber'}, {'name': 'email', 'value': 'bob@bobber.org'}, {'name': 'age', 'value': '21'}]


# 9
## Send Credential

This sends a credential to a holder Openmind-pryvote-voter (Bob), and automates the rest of the protocol. This tutorial works best if you have initialised the [openmind-pryvote-voter notebook](http://127.0.0.1:8889/notebooks/openmind-pryvote-voter.ipynb) and registered the listener for the issue_credential webhook.

There are other ways to issue a credential that require multiple api calls.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [22]:
record = await alice_agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
#extract issuer DID for proof request requirements
issuer_did = record['credential_proposal_dict']['issuer_did']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")


{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'auto_remove': True, 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'fullname', 'value': 'Bob'}, {'name': 'surname', 'value': 'Bobber'}, {'name': 'email', 'value': 'bob@bobber.org'}, {'name': 'age', 'value': '21'}]}, 'connection_id': 'ac91d05a-f26c-487f-8cbc-95c6da689089', 'trace': False, 'comment': '', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:generic_person:0.0.1', 'schema_name': 'generic_person', 'schema_version': '0.0.1', 'schema_issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}
Credential exchange 7c144b2d-d54a-4f9e-9c1f-ed11ac612d85, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 7c144b2d-d54a-4f9e-9c1f-ed11ac612d85, role: issuer, state: offer_sent
Being offered: [{'name': 'fullname', 'value': 'Bob'}, {'name': 'surname', 'value': 'Bobber'}, {'name': 'email', 'value': 'bob@bobber.org'}, 

# 10
## Get Credential Exchange Records

The state of the protocol is stored in an object called a Credential Exchange. These can be retrieved from the agent either individually by ID or the full list.

Each record has a state, representing the stage of the protocol the record is at and a role. Either issuer or holder.

In [23]:
response = await alice_agent_controller.issuer.get_records()
print(response['results'])

[{'initiator': 'self', 'state': 'offer_sent', 'trace': False, 'created_at': '2020-09-01 20:42:17.189447Z', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:generic_person:0.0.1', 'auto_issue': True, 'auto_offer': False, 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:generic_person:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'key_correctness_proof': {'c': '105526218928870020337543973933819360795869252280038683280462362682042055465734', 'xz_cap': '5040606576701625878675463111419867767874097081746988128483384591701907709182860680431011024077149009075672695869230979894346879049168138028031008348995617098170351678632560081271043823459315397452572890602577266502933423338473269093672408253050957456352782247584082759411245933158689530621561595617969223907891838338032640592542058002607174485723301213398949145269573748075049750451766877035690551443415799415796714340988740994465257443209974135956767441108819231522365911553083102213841409950894667555343750264387848556297509

In [24]:
cred_record = await alice_agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Credential exchange 7c144b2d-d54a-4f9e-9c1f-ed11ac612d85, role: issuer, state: offer_sent


# OPTIONAL
## Remove Credential Exchange

It is possible to remove stored credential exchanges from an agent. Typically when you are not interested in the credential being offered, or the credential offer that you have sent has become stale (been ignored?).

In [ ]:
response = await alice_agent_controller.issuer.remove_record(record_id)
print(response)

In [ ]:
response = await alice_agent_controller.issuer.get_records()
print(response)

# 4
# Aries Basic Controller - Bob

This tutorial runs through the issuer-api from the perspective of a holder. It should be run alongside the [openmind-pryvote-community issuer notebook](http://127.0.0.1:8888/notebooks/openmind-pryvote-community.ipynb) from Openmined PryVote Community (Alice)'s perspective.

If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)


In [25]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [26]:
from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

bob_agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

# 5
## Register listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [27]:
loop = asyncio.get_event_loop()
loop.create_task(bob_agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")

cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
bob_agent_controller.register_listeners([cred_listener], defaults=True)

# 11
## Check Credential Exchange Records

The agent will have at least one record if you have run through the [openmind-pryvote-community issuer notebook](http://127.0.0.1:8888/notebooks/openmind-pryvote-community.ipynb) notebook up until send credential.

In [28]:
response = await bob_agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")


Credential exchange ef8b2f10-1979-4587-9a36-40a04d67ea55, role: holder, state: offer_received
Being offered: [{'name': 'fullname', 'value': 'Bob'}, {'name': 'surname', 'value': 'Bobber'}, {'name': 'email', 'value': 'bob@bobber.org'}, {'name': 'age', 'value': '21'}]


# OPTIONAL!
## Propose Credential - not to be executed as part of notebook

If unhappy with the attributes being offered (e.g. there is a mistake), the holder can send a proposal to the issuer for an alternative credential. The holder can also initiate the protocol with a proposal to the issuer, this is left out of scope in the tutorial, but should be possible to edit to achieve this if interested.

Note the proposal will create a new credential exchange record, the tutorial currently does not progress this to issued state but instead continues with the previous credential_exchange.

**Arguments (These can all be found in a credential exchange record)**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [ ]:
### NOT TO BE EXECUTED AS PART OF NOTEBOOK
connection_id = cred_record['connection_id']
schema_id = cred_record['schema_id']
cred_def_id = cred_record['credential_definition_id']
proposed_attributes = [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '0'}]

proposal_record = await bob_agent_controller.issuer.send_proposal(connection_id, schema_id, cred_def_id, proposed_attributes)
proposal_record_id = proposal_record['credential_exchange_id']
proposal_state = proposal_record['state']
proposal_role = proposal_record['role']
proposal_attributes = proposal_record['credential_proposal_dict']['credential_proposal']['attributes']
print(f"Proposal credential exchange record {proposal_record_id}, role: {proposal_role}, state: {proposal_state}")
print(f"Attributes: {attributes}")


# 12
## Request Credential from Alice Issuer

If happy with the attributes being offered in the credential, then the holder requests the credential from the issuer to proceed with the issuance.

It is only possible to request a credential from an exchange when it is in the offer_received state

In [29]:
record = await bob_agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange ef8b2f10-1979-4587-9a36-40a04d67ea55, role: holder, state: request_sent
Handle Credentials
Credential exchange 7c144b2d-d54a-4f9e-9c1f-ed11ac612d85, role: issuer, state: request_received
Attributes: [{'name': 'fullname', 'value': 'Bob'}, {'name': 'surname', 'value': 'Bobber'}, {'name': 'email', 'value': 'bob@bobber.org'}, {'name': 'age', 'value': '21'}]
Handle Credentials
Credential exchange 7c144b2d-d54a-4f9e-9c1f-ed11ac612d85, role: issuer, state: credential_issued
Attributes: [{'name': 'fullname', 'value': 'Bob'}, {'name': 'surname', 'value': 'Bobber'}, {'name': 'email', 'value': 'bob@bobber.org'}, {'name': 'age', 'value': '21'}]


# 13
## Store the credential

Once the Openmined PryVote Community (Alice) issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the credential_received state

In [30]:
record = await bob_agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange ef8b2f10-1979-4587-9a36-40a04d67ea55, role: holder, state: credential_received


In [31]:
response = await bob_agent_controller.issuer.store_credential(cred_ex_id, "My Personal Info Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange ef8b2f10-1979-4587-9a36-40a04d67ea55, role: holder, state: credential_acked
Handle Credentials
Credential exchange 7c144b2d-d54a-4f9e-9c1f-ed11ac612d85, role: issuer, state: credential_acked
Attributes: [{'name': 'fullname', 'value': 'Bob'}, {'name': 'surname', 'value': 'Bobber'}, {'name': 'email', 'value': 'bob@bobber.org'}, {'name': 'age', 'value': '21'}]


# 14
## Send Proof Request
Request a proof of Opemined community member to prove community membership

In [45]:
print("Request proof of Name and Age range from Bob")
#Set some variables

revocation = False
SELF_ATTESTED = True
exchange_tracing = False

#Enable this to ask for attributes to identity a user
req_attrs = [
    {"name": "fullname", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "surname", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "email", "restrictions": [{"issuer_did": issuer_did}]},
#     {"name": "age", "restrictions": [{"issuer_did": issuer_did}]},
]

if revocation:
    req_attrs.append(
        {
            "name": "email",
            "restrictions": [{"issuer_did": agent.did}],
            "non_revoked": {"to": int(time.time() - 1)},
        },
    )
# else:
#     req_attrs.append(
#         {"name": "degree", "restrictions": [{"issuer_did": issuer_did}]}
#     )

if SELF_ATTESTED:
    # test self-attested claims
    req_attrs.append({"name": "country"},)

#Set predicates for Zero Knowledge Proofs
req_preds = [
    # test zero-knowledge proofs
    {
        "name": "age",
        "p_type": "=",
        "p_value": 21,
        "restrictions": [{"issuer_did": issuer_did}],
    }
]

indy_proof_request = {
    "name": "Proof of Personal Information",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}

if revocation:
    indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": alice_connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

Request proof of Name and Age range from Bob


In [46]:
response = await alice_agent_controller.proofs.send_request(proof_request_web_request)
print(response)
alice_presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(alice_presentation_exchange_id)


{'role': 'verifier', 'auto_present': False, 'updated_at': '2020-09-01 20:48:34.620258Z', 'initiator': 'self', 'presentation_exchange_id': '2dd23dc2-48fe-46b2-b182-1e211c1d661b', 'state': 'request_sent', 'connection_id': 'ac91d05a-f26c-487f-8cbc-95c6da689089', 'trace': False, 'thread_id': 'a1e3abad-380a-408f-b523-a6f1a9015813', 'presentation_request': {'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_fullname_uuid': {'name': 'fullname', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_surname_uuid': {'name': 'surname', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_email_uuid': {'name': 'email', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_country_uuid': {'name': 'country'}}, 'requested_predicates': {'0_age_GE_uuid': {'name': 'age', 'p_type': '=', 'p_value': 21, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '881013976097942586932089'}, 'created_at': '2020-09-01 20:48:34.6202

# 15
## Get Proof Request Records

In [47]:
response = await bob_agent_controller.proofs.get_records()
print(response)

print('\n')

state = response['results'][0]["state"]
bob_presentation_exchange_id = response['results'][0]['presentation_exchange_id']
presentation_request = response['results'][0]['presentation_request']

print('Presentation Exchange ID\n')
print(response['results'][0]['presentation_exchange_id'])
pres_x_id = response['results'][0]['presentation_exchange_id']
print('Presentation Request Object\n')
print(response['results'][0]['presentation_request'])
print('Requested Attributes\n')
print(response['results'][0]['presentation_request']['requested_attributes'])
requested_attribs = response['results'][0]['presentation_request']['requested_attributes']
print('Requested Predicates\n')
print(response['results'][0]['presentation_request']['requested_predicates'])
requested_predicates = response['results'][0]['presentation_request']['requested_predicates']

{'results': [{'created_at': '2020-09-01 20:46:29.937794Z', 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'email': '6513909451916333113483947287287970522401646418372523946287247802942771553817', 'fullname': '93006290325627508022776103386395994712401809437930957652111221015872244345185', 'surname': '54306315064670580610056743999040514243692441086237581448976115027979391150265'}, 'a_prime': '2288846259898357248746876285150326329354931754155750818005480616148814675515548084888988951361726660246258270049956629491512825672890113156149007975417828610468120873272024801473444493715712108121545258320187816213062992015629784340878067626375739716099453635881922434782880623183194276333420952942525432220076744774052301660161464895603246921903790679475496323837386250061398005756802186635857063919460198088360514808264997215228318007254353552896774916964006722455646182238628298932481305820673676526064280803034599048608609937676678547451480360548347914747412976

# 16
## Fetch Credentials for Proof Presentation

In [48]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await bob_agent_controller.proofs.get_presentation_credentials(bob_presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }
    else:
        self_attested[referent] = "South Africa"

for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
request = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(request)
print("\nXXX")
print(predicates)
print(revealed)
print(self_attested)

[{'cred_info': {'referent': 'My Personal Info Credential', 'attrs': {'fullname': 'Bob', 'surname': 'Bobber', 'email': 'bob@bobber.org', 'age': '21'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:generic_person:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'rev_reg_id': None, 'cred_rev_id': None}, 'interval': None, 'presentation_referents': ['0_surname_uuid', '0_fullname_uuid', '0_email_uuid']}]

Generate the proof
{'requested_predicates': {}, 'requested_attributes': {'0_fullname_uuid': {'cred_id': 'My Personal Info Credential', 'revealed': True}, '0_surname_uuid': {'cred_id': 'My Personal Info Credential', 'revealed': True}, '0_email_uuid': {'cred_id': 'My Personal Info Credential', 'revealed': True}}, 'self_attested_attributes': {'0_country_uuid': 'South Africa'}}

XXX
{}
{'0_fullname_uuid': {'cred_id': 'My Personal Info Credential', 'revealed': True}, '0_surname_uuid': {'cred_id': 'My Personal Info Credential', 'revealed': True}, '0_email_uuid': {'cred_id': 'My Personal 

# 17
## Send a Proof Presentation

In [49]:
# {
#   "self_attested_attributes": {
#     "additionalProp1": "self_attested_value",
#     "additionalProp2": "self_attested_value",
#     "additionalProp3": "self_attested_value"
#   },
#   "trace": true,
#   "requested_attributes": {
#     "additionalProp1": {
#       "cred_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6",
#       "revealed": true
#     },
#     "additionalProp2": {
#       "cred_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6",
#       "revealed": true
#     },
#     "additionalProp3": {
#       "cred_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6",
#       "revealed": true
#     }
#   },
#   "requested_predicates": {
#     "additionalProp1": {
#       "cred_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6"
#     },
#     "additionalProp2": {
#       "cred_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6"
#     },
#     "additionalProp3": {
#       "cred_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6"
#     }
#   }
# }

proof_presentation = {}



print("\nSend the proof to Openmined Community Agent (Alice)")
send_presentation = await bob_agent_controller.proofs.send_presentation(bob_presentation_exchange_id, request)
print(response)


Send the proof to Openmined Community Agent (Alice)


ClientResponseError: 500, message='Internal Server Error', url=URL('http://bob-agent:8051/present-proof/records/7941be3e-97ca-4be7-add3-68eeac99c4a5/send-presentation')

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/aries_basic_controller/helpers/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/aries_basic_controller/helpers/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/aries_basic_controller/helpers/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/conda/lib/

# 18
## Verify Proof Presentation

In [44]:
verify = await alice_agent_controller.proofs.verify_presentation(alice_presentation_exchange_id)
print(verify)

{'role': 'verifier', 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'email': '6513909451916333113483947287287970522401646418372523946287247802942771553817', 'fullname': '93006290325627508022776103386395994712401809437930957652111221015872244345185', 'surname': '54306315064670580610056743999040514243692441086237581448976115027979391150265'}, 'a_prime': '228884625989835724874687628515032632935493175415575081800548061614881467551554808488898895136172666024625827004995662949151282567289011315614900797541782861046812087327202480147344449371571210812154525832018781621306299201562978434087806762637573971609945363588192243478288062318319427633342095294252543222007674477405230166016146489560324692190379067947549632383738625006139800575680218663585706391946019808836051480826499721522831800725435355289677491696400672245564618223862829893248130582067367652606428080303459904860860993767667854745148036054834791474741297629190021938897843043424352247809566751

# 20
## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [26]:
response = await alice_agent_controller.terminate()
print(response)
response = await bob_agent_controller.terminate()
print(response)

None


NameError: name 'bob_agent_controller' is not defined